<a href="https://colab.research.google.com/github/nicole-malaquias/property_price_analysis/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling

### Importes

In [77]:
import pandas as pd
import numpy as np
from sklearn.metrics import  mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
import pickle
import os
from sklearn.tree import DecisionTreeRegressor # remover no final
from scipy import stats
from sklearn.model_selection import train_test_split

In [78]:
caminho = '/content/drive/MyDrive/Colab Notebooks/desafio/teste_indicium_precificacao.csv'
df = pd.read_csv(caminho)

caminho_turismo = '/content/drive/MyDrive/Colab Notebooks/desafio/New_York_Tourist_Locations.xlsx'
df_t = pd.read_excel(caminho_turismo)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Padronizando nome das variáveis

In [79]:
def rename_columns(df: pd.DataFrame, mapping: dict) -> None:
    """
    Renomeia as colunas de um DataFrame conforme o mapeamento fornecido.

    Parâmetros:
        df (pd.DataFrame): DataFrame que contém as colunas a serem renomeadas.
        mapping (dict): Um dicionário onde as chaves são os nomes atuais das colunas e os valores são os novos nomes das colunas.

    Retorna:
        None: A função modifica o DataFrame fornecido inplace e não retorna nada.

    """
    df.rename(columns=mapping, inplace=True)

mapping = {
    'id': 'id',
    'nome': 'name',
    'host_id': 'host_id',
    'host_name': 'host_name',
    'bairro_group': 'neighborhood_group',
    'bairro': 'neighborhood',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'room_type': 'room_type',
    'price': 'price',
    'minimo_noites': 'minimum_nights',
    'numero_de_reviews': 'number_of_reviews',
    'ultima_review': 'last_review',
    'reviews_por_mes': 'reviews_per_month',
    'calculado_host_listings_count': 'calculated_host_listings_count',
    'disponibilidade_365': 'availability_365'
}

rename_columns(df, mapping)

### Add para reposta quatro



In [80]:
nova_observacao = {'id': [2595],
                   'name': ['Skylit Midtown Castle'],
                   'host_id': [2845],
                   'host_name': ['Jennifer'],
                   'neighborhood_group': ['Manhattan'],
                   'neighborhood': ['Midtown'],
                   'latitude': [40.75362],
                   'longitude': [-73.98377],
                   'room_type': ['Entire home/apt'],
                   'price': [225],
                   'minimum_nights': [1],
                   'number_of_reviews': [45],
                   'last_review': ['2019-05-21'],
                   'reviews_per_month': [0.38],
                   'calculated_host_listings_count': [2],
                   'availability_365': [355]}

nova_observacao_df = pd.DataFrame(nova_observacao)

df = pd.concat([df, nova_observacao_df], ignore_index=True)


### Tratando campos nulos no campo name

In [81]:

def preprocess_text(value):
    """
    Pré-processa um valor de texto removendo espaços em branco adicionais e envolvendo-o em aspas simples,
    se não for nulo.

    Parâmetros:
        value: O valor de texto a ser pré-processado.

    Retorna:
        str: O valor de texto pré-processado, com espaços em branco removidos e envolto em aspas simples,
             se não for nulo.

    Exemplo:
        >>> preprocess_text("  Hello World  ")
        "'Hello World'"
    """
    return str(value).strip() if pd.notnull(value) else ''

df['name'] = df['name'].apply(preprocess_text)


### Removendo outliers

In [83]:
def removed_outlier(df):
    """
    Remove outliers da coluna 'price' de um DataFrame.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna 'price' com os outliers a serem removidos.

    Retorna:
        pd.DataFrame: DataFrame sem os outliers na coluna 'price'.

    Exemplo:
        >>> df = pd.DataFrame({'price': [100, 150, 200, 250, 300, 1000]})
        >>> df_sem_outliers = removed_outlier(df)
        Número de outliers removidos: 1
        >>> print(df_sem_outliers)
           price
        0    100
        1    150
        2    200
        3    250
        4    300
    """
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df_sem_outliers = df[(df['price'] >= limite_inferior) & (df['price'] <= limite_superior)]
    print("Número de outliers removidos:", len(df) - len(df_sem_outliers))
    return df_sem_outliers

df = removed_outlier(df)


Número de outliers removidos: 2972


### Preenchendo quantidade de review por mês

In [39]:
def preencher_reviews_per_month(dataset):
    """
    Preenche os valores faltantes na coluna 'reviews_per_month' com 0 se 'last_review' ou 'number_of_reviews' forem nulos.

    Parâmetros:
        dataset (pd.DataFrame): DataFrame contendo as colunas 'last_review', 'number_of_reviews' e 'reviews_per_month'.

    Retorna:
        pd.DataFrame: DataFrame com os valores faltantes na coluna 'reviews_per_month' preenchidos com 0.

    Exemplo:
        >>> df = pd.DataFrame({'last_review': [None, '2022-01-01', '2022-02-01', None],
        ...                    'number_of_reviews': [10, None, 5, None],
        ...                    'reviews_per_month': [0, 0, 0, 0]})
        >>> df = preencher_reviews_per_month(df)
        >>> print(df)
           last_review  number_of_reviews  reviews_per_month
        0         None               10.0                0.0
        1  2022-01-01                NaN                0.0
        2  2022-02-01                5.0                0.0
        3         None                NaN                0.0
    """
    condicao_vazios = dataset['last_review'].isnull() | dataset['number_of_reviews'].isnull()
    dataset.loc[condicao_vazios, 'reviews_per_month'] = 0
    return dataset

# Exemplo de uso
df = preencher_reviews_per_month(df)


### Adicionando total de ponto turistico por grupo de bairro

In [40]:
def add_tourist_spot_count(dataset, tourist_spot_data):
    """
    Cria o campo tourist_spot_count no dataset de locação.

    Esta função calcula a quantidade de pontos turísticos em cada bairro de Nova York
    e adiciona essa informação como uma nova coluna chamada "tourist_spot_count" ao
    dataset de locação.

    Parâmetros:
    - dataset (DataFrame): O conjunto de dados de locação contendo informações sobre
                           os imóveis em Nova York.
    - tourist_spot_data (DataFrame): O conjunto de dados contendo informações sobre
                                     os pontos turísticos em Nova York, com pelo menos
                                     duas colunas: "neighborhood" (bairro) e
                                     "tourist_spot_count" (quantidade de pontos turísticos).

    Retorna:
    - DataFrame: O dataset de locação com o campo "tourist_spot_count" adicionado.

    Exemplo de Uso:
    >>> dataset = add_tourist_spot_count(dataset, tourist_spot_data)
    """
    borough_count = {'Bronx': 0, 'Brooklyn': 0, 'Manhattan': 0, 'Queens': 0, 'Staten Island': 0}

    boroughs_of_nyc = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

    for address in tourist_spot_data['Address']:
        address_lower = address.lower()
        for borough in boroughs_of_nyc:
            if borough.lower() in address_lower:
                borough_count[borough] += 1

    dataset['tourist_spot_count'] = 0

    for index, row in dataset.iterrows():
        neighborhood_group = row['neighborhood']
        if neighborhood_group in borough_count:
            dataset.at[index, 'tourist_spot_count'] = borough_count[neighborhood_group]
        else:
            # Se o bairro não estiver presente nos dados de pontos turísticos, atribua 0
            dataset.at[index, 'tourist_spot_count'] = 0

    return dataset

df = add_tourist_spot_count(df, df_t)

### Tratando dos campos nulos em price

In [41]:
acomodacoes_zero = df[df['price'] == 0]

for index, row in acomodacoes_zero.iterrows():
    tipo_quarto = row['room_type']

    media_tipo_quarto = df[df['room_type'] == tipo_quarto]['price'].mean()

    df.at[index, 'price'] = media_tipo_quarto

### Dumi de tipo de acomodação


In [82]:
def create_dummy_variables(df):
    """
    Cria variáveis dummy para as colunas 'room_type' e 'neighborhood_group' de um DataFrame.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo as colunas 'room_type' e 'neighborhood_group' para criar variáveis dummy.

    Retorna:
        pd.DataFrame: DataFrame com variáveis dummy adicionadas para 'room_type' e 'neighborhood_group'.

    Exemplo:
        >>> df = pd.DataFrame({'room_type': ['Entire home', 'Private room', 'Shared room'],
        ...                    'neighborhood_group': ['Bronx', 'Manhattan', 'Queens']})
        >>> df = create_dummy_variables(df)
        >>> print(df.columns)
        Index(['room_type', 'neighborhood_group', 'Entire home', 'Private room', 'Shared room', 'Group A', 'Group B', 'Group C'], dtype='object')
    """
    dummies_room_type = pd.get_dummies(df['room_type'])
    df = pd.concat([df, dummies_room_type], axis=1)

    dummies_neighborhood_group = pd.get_dummies(df['neighborhood_group'])
    df = pd.concat([df, dummies_neighborhood_group], axis=1)

    return df


df = create_dummy_variables(df)



# Random Forest

In [43]:
def calcular_indicadores_avaliacao(y_test, y_pred):
    """
    Calculate evaluation metrics for regression.

    Parameters:
    - y_test (array-like): True target values.
    - y_pred (array-like): Predicted target values.

    Returns:
    - tuple: A tuple containing SSE, SST, r_squared, and MAE.
    """
    #  mean squared error (SSE)
    SSE = mean_squared_error(y_test, y_pred) * len(y_test)

    #  total sum of squares (SST)
    SST = ((y_test - y_test.mean()) ** 2).sum()

    #  R-squared
    r_squared = r2_score(y_test, y_pred)

    #  mean absolute error (MAE)
    MAE = mean_absolute_error(y_test, y_pred)

    return SSE, SST, r_squared, MAE

def split_y_x(df):
    y = df['price']
    X = df.drop(['price','name','host_id','host_name','neighborhood','neighborhood_group','room_type','last_review'],axis='columns')
    return y,X

In [ ]:
def random_forest_with_grid(df):
    y, X = split_y_x(df)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_regressor = RandomForestRegressor(n_estimators=10, max_features='sqrt', max_depth=6, min_samples_split=2, random_state=42)

    param_grid = {
        'max_depth': [3, 4, 5, 6],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'ccp_alpha': [0.001, 0.01, 0.1, 1.0]
    }

    grid_search = GridSearchCV(rf_regressor, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring='neg_mean_squared_error')

    grid_search.fit(X_train, y_train)

    best_regressor = grid_search.best_estimator_

    y_pred_best = best_regressor.predict(X_test)

    SSE, SST, r_squared, MAE = calcular_indicadores_avaliacao(y_test, y_pred_best)
    rmse = mean_squared_error(y_test, y_pred_best, squared=False)

    # Salvar os resultados em um arquivo pickle
    with open('random_forest_grid.pkl', 'wb') as f:
        pickle.dump((SSE, SST, r_squared, rmse, MAE, y_pred_best), f)

    return SSE, SST, r_squared, rmse, MAE, y_pred_best


# Gradiant

In [47]:
def gradient_boosting_with_grid(df):
    X = df.drop(['price','name','host_id','host_name','neighborhood','neighborhood_group','room_type','last_review'],axis='columns')
    y = df['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 4, 5]
    }

    gb_regressor = GradientBoostingRegressor(random_state=42)

    grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring='r2')

    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    y_pred_gb_test = best_estimator.predict(X_test)
    r2_score_gb_test = r2_score(y_test, y_pred_gb_test)

    rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb_test))

    mae_gb = mean_absolute_error(y_test, y_pred_gb_test)

    SSE, SST, r_squared, MAE = calcular_indicadores_avaliacao(y_test, y_pred_gb_test)

    with open('gradient_boosting_grid_search_cross.pkl', 'wb') as f:
        pickle.dump((best_params, r2_score_gb_test, rmse_gb, mae_gb, SSE, SST, r_squared, MAE, y_pred_gb_test, grid_search), f)

    return best_params, r2_score_gb_test, rmse_gb, mae_gb, SSE, SST, r_squared, MAE, y_pred_gb_test, grid_search

Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
R2 Score (teste):  0.5778584570772121
RMSE:  43.88060073604842
MAE:  31.4856596353249
SSE: 17685782.90598539, QME: 385.11819580570494
SST: 41895386.044060156, QMT: 912.2963666149893
R-quadrado: 0.5778584570772121
MAE: 31.4856596353249


# Ensemble

In [73]:
def get_base_model_predictions(df):
    y, X = split_y_x(df)

    # Dividir os dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    y_pred_rf = None
    y_pred_gb = None

    if os.path.exists('random_forest_grid.pkl'):
        with open('random_forest_grid.pkl', 'rb') as f:
            _, _, _, _, _, y_pred_rf = pickle.load(f)

    if os.path.exists('gradient_boosting_grid_search_cross.pkl'):
        with open('gradient_boosting_grid_search_cross.pkl', 'rb') as f:
           best_params, r2_score_gb_test, rmse_gb, mae_gb, SSE, SST, r_squared, MAE, y_pred_gb_test, grid_search = pickle.load(f)

    return y_pred_rf, y_pred_gb_test, X_test, y_test


def train_or_load_meta_model(y_pred_rf, y_pred_gb, y_test):

    X_meta = np.column_stack((y_pred_rf, y_pred_gb))

    meta_regressor = GradientBoostingRegressor(random_state=42)
    meta_regressor.fit(X_meta, y_test)

    # Salvar o modelo Gradient Boosting em um arquivo
    with open('gradient_boosting_model_combinado_random.pkl', 'wb') as f:
        pickle.dump(meta_regressor, f)

    return meta_regressor

def predict_with_ensemble(meta_regressor, y_pred_rf, y_pred_gb):
    X_meta = np.column_stack((y_pred_rf, y_pred_gb))
    y_pred_ensemble = meta_regressor.predict(X_meta)

    with open('meta_regressor_predict.pkl', 'wb') as f:
        pickle.dump(y_pred_ensemble, f)

    return y_pred_ensemble

def evaluate_ensemble(y_test, y_pred_ensemble):
    rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
    mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
    r2_score_ensemble = r2_score(y_test, y_pred_ensemble)

    print("RMSE Ensemble: ", rmse_ensemble)
    print("MAE Ensemble: ", mae_ensemble)
    print("R² Score Ensemble: ", r2_score_ensemble)

y_pred_rf, y_pred_gb, X_test, y_test = get_base_model_predictions(df)
meta_regressor = train_or_load_meta_model(y_pred_rf, y_pred_gb, y_test)
y_pred_ensemble = predict_with_ensemble(meta_regressor, y_pred_rf, y_pred_gb)
evaluate_ensemble(y_test, y_pred_ensemble)


RMSE Ensemble:  42.59307291966593
MAE Ensemble:  30.606371764953895
R² Score Ensemble:  0.6022676541666769


In [75]:
import pickle

with open('meta_regressor_predict.pkl', 'rb') as f:
    y_pred_ensemble = pickle.load(f)

observacao_2595_index = df[df['id'] == 2595].index[0]
previsao_2595_ensemble = y_pred_ensemble[observacao_2595_index]

print("Previsão para a observação com ID 2595 pelo modelo de ensemble:", previsao_2595_ensemble)


Previsão para a observação com ID 2595 pelo modelo de ensemble: 180.56829960997138


# Pergunta 3

 Explique como você faria a previsão do preço a partir dos dados. Quais
variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de
problema estamos resolvendo (regressão, classificação)? Qual modelo
melhor se aproxima dos dados e quais seus prós e contras? Qual medida de
performance do modelo foi escolhida e por quê?

Para prever o preço a partir dos dados em questão, foi inicialmente implementado o procedimento 'dummy' nas variáveis `neighborhood_group` e `room_type`. Esta implementação possibilitou a conversão dessas variáveis categóricas em numéricas. Ademais, criou-se uma variável chamada `tourist_spot_count` com o objetivo de quantificar os pontos turísticos em cada `neighborhood_group`. Esta variável foi obtida através da soma dos pontos turísticos pertencentes a cada `neighborhood_group` em um conjunto de dados adicional. Adicionalmente, outliers foram removidos e normalizamos dados de preços que estavam com valor zero. Isso porque se uma acomodação está cadastrada em um site de aluguel, é pouco provável que seja gratuita. Portanto, normalizamos o valor para a média do tipo de acomodação.

Enquadramos o problema como uma questão de regressão, dado que a variável 'price' é uma variável quantitativa contínua. Para facilitar essa abordagem, as variáveis categóricas foram removidas.

O Modelo de Regressão Ensemble apresentou a melhor performance nos testes, sendo o mais adequado para se aproximar dos dados. Este modelo é uma combinação de dois modelos distintos: Random Forest e Gradient Boosting. Cada um destes modelos foi treinado individualmente e, posteriormente, foram combinados por meio de um meta-modelo, o Gradient Boosting Regressor.

A performance do modelo foi avaliada através do RMSE (Root Mean Square Error), MAE (Mean Absolute Error) e R² (R-squared). Estas métricas foram escolhidas devido à sua capacidade de fornecer uma avaliação abrangente do desempenho do modelo de regressão. O RMSE e o MAE medem a precisão das previsões em termos de erros absolutos, enquanto que o R² indica o quanto o modelo é capaz de explicar a variabilidade dos dados. A combinação destes indicadores permite uma avaliação holística do desempenho do modelo de regressão.

Os resultados finais do modelo de regressão ensemble foram:

- RMSE Ensemble: 42.593
- MAE Ensemble: 30.606
- R² Score Ensemble: 0.602

# Pergunta 4

Supondo um apartamento com as seguintes características:

{'id': 2595,
'nome': 'Skylit Midtown Castle',
'host_id': 2845,
'host_name': 'Jennifer',
'bairro_group': 'Manhattan',
'bairro': 'Midtown',
'latitude': 40.75362,
'longitude': -73.98377,
'room_type': 'Entire home/apt',
'price': 225,
'minimo_noites': 1,
'numero_de_reviews': 45,
'ultima_review': '2019-05-21',
'reviews_por_mes': 0.38,
'calculado_host_listings_count': 2,
'disponibilidade_365': 355}


Assim, a previsão para a observação com ID 2595 pelo nosso modelo de regressão ensemble foi de $180.57.